In [11]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns; sns.set()

from scipy.stats import gaussian_kde, norm 
from empiricaldist import Pmf

ModuleNotFoundError: No module named 'empiricaldist'

# Prior 

In [ ]:
def read_data(csv_fname): 
    """Read the showcase price data.
    """
    
    df = pd.read_csv(csv_fname, index_col=0, skiprows=[1])
    return df.dropna().transpose()

In [ ]:
csv_fname_2011 = '../data/showcases.2011.csv'
csv_fname_2012 = '../data/showcases.2012.csv'

df_2011 = read_data(csv_fname_2011)
df_2012 = read_data(csv_fname_2012)

In [ ]:
df_2011.head()

In [ ]:
df_2012.head()

In [ ]:
df_2011.info()

In [ ]:
df_2012.info()

In [ ]:
df = pd.concat([df_2011, df_2012], ignore_index=True)
df.head()

In [ ]:
df.info()

# Kernel density estimation

Ta coi dữ liệu ở trên là mẫu từ quần thể các mức giá có thể có. KDE sử dụng mẫu để ước lượng một phân phối mượt.

In [ ]:
def kde_from_sample(sample, qs): 
    """Make a kernel density estimate from a sample.
    
    Params: 
    - qs: a sequence of quantiles.
    """
    
    kde = gaussian_kde(sample)
    probs = kde(qs)
    
    pmf = Pmf(probs, qs)          # tạo pmf từ mảng qs và probs tương ứng 
    pmf.normalize()
    
    return pmf 

In [ ]:
qs = np.linspace(0, 80_000, 81)    # start_val, stop_val, num 
prior_1 = kde_from_sample(df['Showcase 1'], qs)
prior_2 = kde_from_sample(df['Showcase 2'], qs)

plt.figure(figsize=(7, 5))
plt.plot(prior_1.qs, prior_1.ps, linestyle='--', color='r')
plt.plot(prior_2.qs, prior_2.ps, color='b')
plt.xlabel('Showcase value ($)')
plt.ylabel('pmf')
plt.legend(['Prior 1', 'Prior 2'])
plt.title('Prior distribution of showcase value')
plt.show()

# Distribution of error 

In [ ]:
sample_diff_1 = df['Bid 1'] - df['Showcase 1']
sample_diff_2 = df['Bid 2'] - df['Showcase 2']

# visualize the distribution of these differences using KDE 
qs = np.linspace(-40_000, 20_000, 61)
kde_diff_1 = kde_from_sample(sample_diff_1, qs) 
kde_diff_2 = kde_from_sample(sample_diff_2, qs)

plt.figure(figsize=(7, 5))
plt.plot(kde_diff_1.qs, kde_diff_1.ps, linestyle='--', color='r')
plt.plot(kde_diff_2.qs, kde_diff_2.ps, linestyle='-', color='b')
plt.xlabel('Difference in value (guess - actual) ($)')
plt.ylabel('pmf')
plt.legend(['Diff 1', 'Diff 2'])
plt.title('Difference between bid and actual value')
plt.show()

Nhận xét: 
- Nhìn vao hình vẽ trên, ta thấy đa phần mọi người đoán giá thấp hơn so với giá thực. Điều này là hợp lý vì mọi người đang làm theo thể lệ của cuộc chơi. Đoán cao hơn là thua luôn. Nên mọi người chủ động đưa ra giá bid thấp hơn giá dự đoán của họ.
- Ví dụ: nếu họ dự đoán giá của showcase là $40_000, họ sẽ bid $36_000 để tránh giá cao hơn so với giá thực. 
- Phân phối trên khá giống với phân phối chuẩn (mặc dù đuôi trái dài hơn đuôi phải), vì thế ta sẽ tóm tắt nó bằng mean và std.

In [ ]:
mean_diff_1 = sample_diff_1.mean()
std_diff_1 = sample_diff_1.std()

mean_diff_1, std_diff_1

In [ ]:
error_dist_1 = norm(0, std_diff_1)    # normal distribution

error = -100
error_dist_1.pdf(error)               # prob density for error of -100 

Nhận xét: Nhìn vào std_diff_1 ở trên, mean 0, thì khi thấy prob density của sai số -100 rất bé, ta thấy khá bất ngờ. Nhưng có hai cách giải thích sau đây: 
- Một là, đây là prob density chứ kh phải prob, nên prob density chắc chắn nhỏ hơn prob.
- Thứ hai, phân phối chuẩn ở trên rất rộng nên prob density bị dàn trải hơn nên prob density sẽ nhỏ hơn, chứ so với các giá trị khác thì sai số -100 vẫn lớn hơn nhiều. 
- Nhìn vào biểu đồ ở dưới thì ta đã chắn chắn là -100 có khả năng xảy ra lớn hơn các giá trị khác. 

In [ ]:
errors = np.arange(-25_000, 25_000)
plt.figure(figsize=(10, 5)) 
plt.plot(errors, error_dist_1.pdf(errors))
plt.axvline(x=-100, color='r', linestyle='--', lw=1)
plt.xlabel('errors ($)')
plt.ylabel('pdf')
plt.legend(['Error 1', 'Error = -100'])
plt.title('Error distribution of sample 1')
plt.show()